# DS-SF-36 | Unit Project | 1 | Research Design | Starter Code

In this first unit project, you will create a framework to scope out data science projects.  This framework will provide you with a guide to develop a well-articulated problem statement and analysis plan that will be robust and reproducible.

## Part A.  Evaluate the following problem statement:

> Determine which factors most impact how many average check-ins a restaurants gets. How do we predict how many visitors a restaurant is going to get? I will use the venues API (https://developer.foursquare.com/docs/responses/venue) from foursquare to regress the number of photos, whether a menu is available, sentiment analysis of recent reviews, price, sentiment analysis of key phrases to predict the average number of checkins per week (calc by using total checkins divided by number of weeks since the venue was created on foursquare).  The period would be a list of all restaurants in San Francisco on Foursquare which we can segment into test and training data

> ### Question 1.  What is the outcome?

Answer: The outcome is a prediction of how many check-ins to expect in a week given the various inputs

> ### Question 2.  What are the predictors/covariates?

Answer: The number of photos, whether a menu is available, sentiment analysis of recent reviews, price, sentiment analysis of key phrases

> ### Question 3.  What timeframe is this data relevent for?

Answer: San Francisco restaurants that are on Foursquare in July 2017

> ### Question 4.  What is the hypothesis?

Answer: Foot traffic is correlated with the depth of the online profile

## Part B.  Let's start exploring our Foursquare dataset and answer some simple questions:

In [6]:
import os
import pandas as pd


pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)





> ### Question 5.  Create a data dictionary.

Answer: TODO

(Use the template below)

Variable | Description | Type of Variable
---|---|---
Is Menu available | 0 = No, 1 = Yes | Categorical
Number of Photos | Integer | Continuous
Rating | Integer | Continuous
Reviews Polarity | Integer | Continuous
Price | Integer | Continuous


We would like to explore the association between X and Y. 

> ### Question 6.  What is the outcome?

Answer: Outcome is the number of check-ins per week

## Part C.  Create an exploratory analysis plan by answering the following questions:

Because the answers to these questions haven't yet been covered in class yet, this section is optional.  This is by design.  By having you guess or look around for these answers will help make sense once we cover this material in class.  You will not be penalized for wrong answers but we encourage you to give it a try!

> ### Question 11. What are the goals of the exploratory analysis?

Answer: 

* Summarize the main characteristics of the dataset (looking at summary statistics)
* Visualize the data set 
* Formulate additional hypotheses
* Formulate assumptions and edge cases


> ### Question 12.  What are the assumptions of the distribution of data?

Answer: The assumption is that the data for checkins is normally distributed. There are probably 1-2 major factors that define the curve

> ### Question 13.  How will determine the distribution of your data?

Answer: TODO

> ### Question 14.  How might outliers impact your analysis?

Answer: TODO

> ### Question 15.  How will you test for outliers?

Answer: TODO

> ### Question 16.  What is colinearity?

Answer: TODO

> ### Question 17.  How will you test for covariance?

Answer: TODO

> ### Question 18.  What is your exploratory analysis plan?

> Using the above information, write an exploratory analysis plan that would allow you or a colleague to reproduce your analysis one year from now.

Answer: TODO